## Homework

In this homework, we will use Credit Card Data from [the previous homework](https://github.com/alexeygrigorev/mlbookcamp-code/blob/master/course-zoomcamp/cohorts/2022/04-evaluation/homework.md).

## Question 1

* Install Pipenv
* What's the version of pipenv you installed?
* Use `--version` to find out

### Solution
Run in your terminal:

1. pip install pipenv
2. pipenv install numpy pandas scikit-learn==1.0.2
3. pipenv --version
4. pipenv shell

In [38]:
!pipenv --version

pipenv, version 2020.11.15



## Question 2

* Use Pipenv to install Scikit-Learn version 1.0.2
* What's the first hash for scikit-learn you get in Pipfile.lock?

Answer: sha256:08ef968f6b72033c16c479c966bf37ccd49b06ea91b765e1cc27afefe723920b

## Models

An already trained classification model will be used. The next step is to download the model and an associated DictVectorized to this task.
Download the following files.

In [2]:
PREFIX='https://raw.githubusercontent.com/alexeygrigorev/mlbookcamp-code/master/course-zoomcamp/cohorts/2022/05-deployment/homework'

In [3]:
!python install wget
!python -m wget $PREFIX/model1.bin
!python -m wget $PREFIX/dv.bin


Saved under model1.bin

Saved under dv.bin


## Serve four flask app

Before continuing, you must run flask app. Execute:

`gunicorn --bind=0.0.0.0:9696 service:app` -> Production (Linux)
`waitress-serve --listen=0.0.0.0:9696 service:app` -> Production (Windows)

`python service.py` -> Development


## Question 3

Let's use these models!

* Write a script for loading these models with pickle
* Score this client:

```json
{"reports": 0, "share": 0.001694, "expenditure": 0.12, "owner": "yes"}
```

What's the probability that this client will get a credit card? 

* 0.162
* 0.391
* 0.601
* 0.993


NOTE: Run `request_test.py` for check the solution

Answer: 0.162

In [39]:
import requests
def send_request(data):   
    url = 'http://localhost:9696/predict'
   
    client_id = 'xyz-123'

    response = requests.post(url, json=data).json()

    print(response)
    
    if response['is_credit'] == True:
        print('Customer %s will get a credit' % client_id)
    else:
        print('Customer %s will not get a credit' % client_id)

In [40]:
client ={"reports": 0, "share": 0.001694, "expenditure": 0.12, "owner": "yes"}
data ={"client":client,"model":'model1.bin'}
send_request(data)

{'credit_probability': 0.16213414434326598, 'is_credit': False}
Customer xyz-123 will not get a credit


## Question 4

Now let's serve this model as a web service

* Install Flask and gunicorn (or waitress, if you're on Windows)
* Write Flask code for serving the model
* Now score this client using `requests`:

In [41]:
client ={"reports": 0, "share": 0.245, "expenditure": 3.438, "owner": "yes"}
data ={"client":client,"model":'model1.bin'}
send_request(data)

{'credit_probability': 0.9282218018527452, 'is_credit': True}
Customer xyz-123 will get a credit


## Docker

Install [Docker](https://github.com/alexeygrigorev/mlbookcamp-code/blob/master/course-zoomcamp/05-deployment/06-docker.md). We will use it for the next two questions.

For these questions, we prepared a base image: `svizor/zoomcamp-model:3.9.12-slim`. 
You'll need to use it (see Question 5 for an example).

This image is based on `python:3.9.12-slim` and has a logistic regression model 
(a different one) as well a dictionary vectorizer inside. 

**1. Run in terminal**

`docker pull svizor/zoomcamp-model:3.9.12-slim`

## Question 5
So what's the size of this base image?

* 15 Mb
* 125 Mb
* 275 Mb
* 415 Mb

In [11]:
!docker images

REPOSITORY              TAG           IMAGE ID       CREATED      SIZE
svizor/zoomcamp-model   3.9.12-slim   571a6fdc554b   7 days ago   125MB


Docker commands
* Build docker dependencies
    -    docker build -t zoomcamp-model .
* Open docker bash
    -    docker run -it --rm --entrypoint=bash zoomcamp-model
* Run docker
    -    docker run -it --rm -p 9696:9696 zoomcamp-model

## Question 6

Let's run your docker container!

After running it, score this client once again:

In [36]:
client={"reports": 0, "share": 0.245, "expenditure": 3.438, "owner": "yes"}
data ={"client":client,"model":'model2.bin'}
send_request(data)

{'credit_probability': 0.7692649226628628, 'is_credit': True}
Customer xyz-123 will get a credit
